In [ ]:
## https://www.kaggle.com/lostbox/rsna-classification-87-6-best-accuracy-p-cda585 was used as a guide in the transformation process

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from pydicom import dcmread
from tqdm import tqdm
from torch.utils import data
from sklearn.model_selection import train_test_split

## Preparing labels

In [ ]:
label_data = pd.read_csv('../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
columns = ['patientId', 'Target']

label_data = label_data.filter(columns)
label_data.head(5)

In [ ]:
train_labels, val_labels = train_test_split(label_data.values, test_size=0.1)
print(train_labels.shape)
print(val_labels.shape)

In [ ]:
train_f = '../input/rsna-pneumonia-detection-challenge/stage_2_train_images'
test_f = '../input/rsna-pneumonia-detection-challenge/stage_2_test_images'

train_paths = [os.path.join(train_f, image[0]) for image in train_labels]
val_paths = [os.path.join(train_f, image[0]) for image in val_labels]


In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor()])

## Write a custom dataset 

In [ ]:
class Dataset(data.Dataset):
    
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform
    
    def __getitem__(self, index):
        image = dcmread(f'{self.paths[index]}.dcm')
        image = image.pixel_array
        image = image / 255.0

        image = (255*image).clip(0, 255).astype(np.uint8)
        image = Image.fromarray(image).convert('RGB')

        label = self.labels[index][1]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label
    
    def __len__(self):
        
        return len(self.paths)

In [ ]:
train_dataset = Dataset(train_paths, train_labels, transform=transform)
image = iter(train_dataset)
img, label = next(image)
print(f'Tensor:{img}, Label:{label}')
img = np.transpose(img, (1, 2, 0))
plt.imshow(img)

## Train image shape

In [ ]:
img.shape

## Prepare training and validation dataloader

In [ ]:
train_dataset = Dataset(train_paths, train_labels, transform=transform)
val_dataset = Dataset(val_paths, val_labels, transform=transform)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=128, shuffle=False)

## Check dataloader

In [ ]:
batch = iter(train_loader)
images, labels = next(batch)

image_grid = torchvision.utils.make_grid(images[:4])
image_np = image_grid.numpy()
img = np.transpose(image_np, (1, 2, 0))
plt.imshow(img)

## Specify device object

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

## Load pre-trained ResNet18 and fine-tune

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 2)

model.to(device)

criterion = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
print(model) # ResNet18 Model

## Write a train code and RUN

In [ ]:
num_epochs = 20
# Train the model
total_step = len(train_loader)

train_loss, validation_loss = [], []
train_acc, validation_acc = [], []

for epoch in range(num_epochs):
    model.train()
    
    running_loss = 0.
    correct, total = 0, 0
    steps = 0
    # Training step
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        print("Loss: " + str(loss.item()))
        
        steps += 1
        running_loss += loss.item()
                
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # get accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        
        if (i+1) % 2000 == 0:
            
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    train_loss.append(running_loss/len(train_loader))
    train_acc.append(correct/total)
    
    print(f'Epoch: {epoch + 1},  Training Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {100*correct/total: .2f}%')
    
    # Validation step
    model.eval()
    running_loss = 0.
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)
            predictions = model(images)
            loss = criterion(predictions, labels)
            running_loss += loss.item()
            _, predicted = torch.max(predictions, 1)
            total += labels.size(0)
            correct += (labels == predicted).sum()
    validation_loss.append(running_loss/len(val_loader))
    validation_acc.append(correct/total)
        
    print(f'Epoch: {epoch+1}/{num_epochs}, Validation Loss : {running_loss / len(val_loader)} Val_Acc: {100*correct/total}')

In [ ]:
train_loss

In [ ]:
train_acc

In [ ]:
validation_loss

In [ ]:
validation_acc

## Test model

In [ ]:
model.eval()

correct = 0
total = 0
val_preds = torch.tensor([]).to(device)
for images, labels in tqdm(val_loader):
    images = images.to(device)
    labels = labels.to(device)
    predictions = model(images)
    _, predicted = torch.max(predictions, 1)
    val_preds = torch.cat((val_preds, predicted),dim=0)
    total += labels.size(0)
    correct += (labels == predicted).sum()
print(f'Val_Acc: {100*correct/total}')

In [ ]:
val_preds.shape

In [ ]:
pip install git+git://github.com/raghakot/keras-vis.git --upgrade --no-deps

In [ ]:
def print_saliency(i):
    image_test = images[i]
    # from: https://towardsdatascience.com/saliency-map-using-pytorch-68270fe45e80
    image_test = image_test.reshape(1, 3, 224, 224)
    image_test = image_test.to(device)
    image_test.requires_grad_()

    # Retrieve output from the image
    output = model(image_test)

    # Catch the output
    output_idx = output.argmax()
    output_max = output[0, output_idx]

    # Do backpropagation to get the derivative of the output based on the image
    output_max.backward()
    
    # Retireve the saliency map and also pick the maximum value from channels on each pixel.
    # In this case, we look at dim=1. Recall the shape (batch_size, channel, width, height)
    saliency, _ = torch.max(image_test.grad.data.abs(), dim=1) 
    saliency = saliency.reshape(224, 224)

    # Reshape the image
    image_test = image_test.reshape(-1, 224, 224)

    # Visualize the image and the saliency map
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(image_test.cpu().detach().numpy().transpose(1, 2, 0))
    ax[0].axis('off')
    ax[1].imshow(saliency.cpu(), cmap='hot')
    ax[1].axis('off')
    plt.tight_layout()
    fig.suptitle('The Image and Its Saliency Map')
    plt.show()


In [ ]:
print_saliency(0)

In [ ]:
print(labels[0])

In [ ]:
print_saliency(50)

In [ ]:
print(labels[50])

In [ ]:
print_saliency(78)

In [ ]:
print(labels[78])

In [ ]:
print_saliency(20)

In [ ]:
print(labels[20])

In [ ]:
torch.save(model.state_dict(), "Transfer_Learning_Resnet.pt")

# Confusion Matrix
# 

In [ ]:
model.load_state_dict(torch.load('../input/transfer-learning-resnetpt/Transfer_Learning_Resnet.pt'))

In [ ]:
model.eval()

correct = 0
total = 0
val_preds = torch.tensor([]).to(device)
val_labels = torch.tensor([]).to(device)
for images, labels in tqdm(val_loader):
    images = images.to(device)
    labels = labels.to(device)
    predictions = model(images)
    _, predicted = torch.max(predictions, 1)
    val_preds = torch.cat((val_preds, predicted),dim=0)
    val_labels = torch.cat((val_labels, labels),dim=0)
    total += labels.size(0)
    correct += (labels == predicted).sum()
print(f'Val_Acc: {100*correct/total}') # Acc may be messed up because train split may be diff this time around

In [ ]:
print(val_preds.shape, val_labels.shape)

In [ ]:
model.eval()

correct = 0
total = 0
train_preds = torch.tensor([]).to(device)
train_labels = torch.tensor([]).to(device)
for images, labels in tqdm(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    predictions = model(images)
    _, predicted = torch.max(predictions, 1)
    train_preds = torch.cat((train_preds, predicted),dim=0)
    train_labels = torch.cat((train_labels, labels),dim=0)
    total += labels.size(0)
    correct += (labels == predicted).sum()
print(f'Train_Acc: {100*correct/total}') # Acc may be messed up because train split may be diff this time around

In [ ]:
train_preds.shape

In [ ]:
preds = torch.cat((train_preds, val_preds), dim = 0)
labels = torch.cat((train_labels, val_labels), dim = 0)
print(preds.shape, labels.shape)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
# def plot_confusion_matrix(cm,channels, title='Confusion matrix', cmap=plt.cm.Blues,filename="confusion_matrix.png"):
#     #plt.figure()
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(channels))
#     plt.xticks(tick_marks, channels, rotation=45,ha='right')
#     plt.yticks(tick_marks, channels)
#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.savefig("data/"+filename)
#     plt.show()

In [ ]:
labels.cpu()

In [ ]:
cm = confusion_matrix(labels.cpu(), preds.cpu())
print(type(cm))
cm


In [ ]:
cm/sum(cm)

In [ ]:
classes = np.array(["Negative", "Positive"])

In [ ]:
df_cm = pd.DataFrame(cm/cm.sum(), classes, classes)
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, fmt='.2%', annot_kws={"size": 16}, cmap='Blues') # font size
plt.xlabel("Predicted")
plt.ylabel("Actual")

plt.show()
